In [16]:
## RAG Application With GPT-4o

In [1]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 6


# 1. Get a Data Loader


In [2]:
from langchain_community.document_loaders import WebBaseLoader


In [3]:
loader = WebBaseLoader("https://www.scit.edu/mba-in-data-sciences-and-data-analytics")
data = loader.load()
data

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMBA in Data Science and Data Analytics in India - SCIT, Pune\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n About \n\nThe Institute\nSCIT Management\nAbout Symbiosis\nSymbiosis Board\n\nAwards & Achievements\nSCIT Campus\nSports & Fitness\nAbout Staff\n\n\n Programmes \n\nMBA (ITBM)\nMBA (DS & DA)\nMBA (Executive) IT(Programme discontinued from A.Y. 2021-2022)\nPh. D. (SIU)\n\n\n Placements \n\nWhy SCIT?\n Placement Stats\nStudents Profile\n Recruiter\'s List\n\n\n Corporate \n\nTraining Programmes\n Collaborations\nCorporate Comments\n\n\n Faculty & Research \n\nFaculty Recognition\nFaculty Members\nResearch @ SCIT\nResearch Policy\nPublications\nResearch Projects\nCase Research\nExtension Activity\n\n\n Students \n\nStudent Achievements\nStudent Activities\nBest Practices\nAcademic Bank of Credits (ABC) \nGender Equalit

# 2. Convert data to Vector Database


In [4]:

from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

In [6]:
documents

[Document(page_content="MBA in Data Science and Data Analytics in India - SCIT, Pune\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n About \n\nThe Institute\nSCIT Management\nAbout Symbiosis\nSymbiosis Board\n\nAwards & Achievements\nSCIT Campus\nSports & Fitness\nAbout Staff\n\n\n Programmes \n\nMBA (ITBM)\nMBA (DS & DA)\nMBA (Executive) IT(Programme discontinued from A.Y. 2021-2022)\nPh. D. (SIU)\n\n\n Placements \n\nWhy SCIT?\n Placement Stats\nStudents Profile\n Recruiter's List\n\n\n Corporate \n\nTraining Programmes\n Collaborations\nCorporate Comments\n\n\n Faculty & Research \n\nFaculty Recognition\nFaculty Members\nResearch @ SCIT\nResearch Policy\nPublications\nResearch Projects\nCase Research\nExtension Activity\n\n\n Students \n\nStudent Achievements\nStudent Activities\nBest Practices\nAcademic Bank of Credits (ABC) \nGender Equality Workshops\nEvents @ SCIT\nCore Committe

In [7]:
from langchain_openai import OpenAIEmbeddings
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)
vector

# 3. Make a RAG pipeline


In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [9]:
llm = ChatOpenAI(model="gpt-4o") ## Calling Gpt-4o
prompt = hub.pull("rlm/rag-prompt")
prompt



ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [10]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [11]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [18]:
question = "What is SCIT"
result = qa_chain({"query": question })
result

{'query': 'What is SCIT',
 'result': 'SCIT stands for Symbiosis Centre for Information Technology, which is a constituent of Symbiosis International University in Pune, India. It offers specialized MBA programs, notably in Data Sciences and Data Analytics (DS&DA), among others. Admissions to SCIT are based on the SNAP test, followed by a Group Exercise and Personal Interview process.'}

In [19]:
result["result"]

'SCIT stands for Symbiosis Centre for Information Technology, which is a constituent of Symbiosis International University in Pune, India. It offers specialized MBA programs, notably in Data Sciences and Data Analytics (DS&DA), among others. Admissions to SCIT are based on the SNAP test, followed by a Group Exercise and Personal Interview process.'

In [20]:
import pprint
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(result["result"])

('SCIT stands for Symbiosis Centre for Information Technology, which is a '
 'constituent of Symbiosis International University in Pune, India. It offers '
 'specialized MBA programs, notably in Data Sciences and Data Analytics '
 '(DS&DA), among others. Admissions to SCIT are based on the SNAP test, '
 'followed by a Group Exercise and Personal Interview process.')


In [23]:
question = "Explain me about the subjects in DSDA"
result = qa_chain({"query": question })
result

{'query': 'Explain me about the subjects in DSDA',
 'result': 'The subjects in the MBA in Data Sciences and Data Analytics (DSDA) program include core courses like Data Preparation and Data Management, Network Security, Data Mining, Programming for Data Sciences, and Predictive Analytics. Electives cover areas such as Big Data Analytics, Marketing Analytics, and Advanced R Programming. The curriculum aims to prepare students for business and techno-functional roles in data sciences and analytics projects, offering hands-on experience with tools like Python, Hadoop, R, Tableau, and IBM Watson.'}